# Ejercicio 5-3 Añade la habilidad de usar markowitz a los monos

In [1]:
import pandas as pd
import time
import numpy as np
import random
from tqdm import tqdm

In [3]:
def calcular_mono():
    
    # Extracción fechas y activos
    fecha_compra_mono = random.sample(range(ventana_mark+1, len(rent_activos) - min_dias_mono), 1)[0]
    fecha_venta_mono = random.sample(range(fecha_compra_mono+min_dias_mono, len(rent_activos)), 1)[0]
    activos_seleccionados_mono = random.sample(activos_disponibles, num_activos_mono)
    
    # Extracción de periodo para procesamiento de la ventana
    activos_mono = benchmark_global.iloc[fecha_compra_mono-ventana_mark: fecha_compra_mono,activos_seleccionados_mono]
    rent_activos_mono = rent_activos.iloc[fecha_compra_mono-ventana_mark: fecha_compra_mono,activos_seleccionados_mono]
    
    # Cálculo de rendimientos de cada combinación de pesos
    matriz_var_covarianzas_mono = rent_activos_mono.cov()
    rent_diaria_mono = (np.log(activos_mono.iloc[0]) - np.log(activos_mono.iloc[-1])) / len(activos_mono)
    rand_matrix = np.random.randint(0,100,(num_simulaciones_mark_mono,num_activos_mono))
    pesos_mono = pd.DataFrame(rand_matrix / rand_matrix.sum(axis=1)[:, np.newaxis], columns=activos_mono.columns)
    rentabilidad_carteras_mono = (pesos_mono * rent_diaria_mono).sum(axis=1)
    matriz_intermedia = pesos_mono.dot(matriz_var_covarianzas_mono)
    matriz_intermedia = matriz_intermedia * pesos_mono
    riesgo_carteras_mono = matriz_intermedia.sum(axis=1) ** 0.5
    eficiencia_carteras_mono = rentabilidad_carteras_mono / riesgo_carteras_mono
    pesos_mono['eficiencia'] = eficiencia_carteras_mono
    
    # Extracción de cartera con mayor eficiencia
    max_eficiencia = pesos_mono.sort_values(by='eficiencia', ascending=False).iloc[0,:]
    resultado_mono = [0] * len(benchmark_global.columns)
    dict_replacement = {a:b for a,b in zip(activos_seleccionados_mono, max_eficiencia)}
    for k, v in dict_replacement.items():
        resultado_mono[k] = v
    
    return resultado_mono, fecha_compra_mono, fecha_venta_mono

In [5]:
start_time = time.time()

num_simulaciones = 50000 
num_activos_mono = 5
ventana_mark = 240
num_simulaciones_mark_mono = 1000
min_dias_mono = 2
dinero = 1000000

np.random.seed(42)

DAX = pd.read_csv("DAX.csv", sep=';')
DAX.drop(columns=['Dax','BUND'], axis=1, inplace=True)
DAX.set_index('Fecha', inplace=True)
div_EUR = pd.DataFrame([1] * len(DAX), columns=['div_EUR'])

FTSE = pd.read_csv("FTSE.csv", sep=';')
FTSE.set_index('Fecha', inplace=True)
div_GBP = FTSE['GBP']
FTSE.drop(columns=['GBP'], axis=1, inplace=True)
FTSE = FTSE.mul(div_GBP, axis=0)

NASDAQ = pd.read_csv("NASDAQ.csv", sep=';')
NASDAQ.set_index('Fecha', inplace=True)
div_USD = NASDAQ['USD']
NASDAQ.drop(columns=['USD'], axis=1, inplace=True)
NASDAQ = NASDAQ.mul(div_USD, axis=0)

HANGSENG = pd.read_csv("HANG SENG.csv", sep=';')
HANGSENG.set_index('Fecha', inplace=True)
div_HKD = HANGSENG['HKD']
HANGSENG.drop(columns=['HKD'], axis=1, inplace=True)
HANGSENG = HANGSENG.mul(div_HKD, axis=0)

BOVESPA = pd.read_csv("BOVESPA.csv", sep=';')
BOVESPA.set_index('Fecha', inplace=True)
div_BRL = BOVESPA['BRL']
BOVESPA.drop(columns=['BRL'], axis=1, inplace=True)
BOVESPA = BOVESPA.mul(div_BRL, axis=0)

benchmark_global = DAX.copy()
list_df = [FTSE, NASDAQ, HANGSENG, BOVESPA]

for df in list_df:
    benchmark_global = benchmark_global.join(df) 
    
rent_activos = np.log(benchmark_global).diff()

activos_disponibles = range(len(rent_activos.columns))

fecha_compra, fecha_venta, pesos = [], [], []

for simulacion in tqdm(range(num_simulaciones)):
    peso_local, fc_local, fv_local = calcular_mono()
    fecha_compra.append(fc_local)
    fecha_venta.append(fv_local)
    pesos.append(peso_local)
    
pesos = pd.DataFrame(pesos, columns=benchmark_global.columns)
precios_compra = benchmark_global.iloc[fecha_compra]
precios_venta = benchmark_global.iloc[fecha_venta]
num_acciones = dinero * pesos.values / precios_compra 
rdo_accion = num_acciones * (precios_venta.values - precios_compra.values)
rdo_simulacion = rdo_accion.sum(axis=1)

pesos['rdo_simulacion'] = rdo_simulacion.values
pesos['fecha_compra'] = precios_compra.index
pesos['fecha_venta'] = precios_venta.index
pesos['rentabilidad'] = pesos.apply(lambda row: np.log(row['rdo_simulacion'] + dinero) -np.log(dinero) , axis=1)

mejor_simulacion = pesos.sort_values(by='rentabilidad', ascending=False).iloc[0,:]
cuantiles = pesos['rentabilidad'].quantile([0, .1, .2, .3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])

print("--- %s seconds ---" % (time.time() - start_time))

100%|██████████| 50000/50000 [04:30<00:00, 185.02it/s]


--- 275.6870241165161 seconds ---


In [6]:
cuantiles

0.0   -1.869962
0.1   -0.232146
0.2   -0.092545
0.3   -0.019083
0.4    0.033864
0.5    0.098358
0.6    0.182844
0.7    0.290924
0.8    0.447515
0.9    0.713604
1.0    3.849289
Name: rentabilidad, dtype: float64

## Otra manera de hacerlo

In [2]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import os
from random import sample
import multiprocessing as mp
import itertools
from functools import partial

In [3]:
# Variables
start_time = time.time()

dax_filename = 'DAX.csv'
ftse_filename = 'FTSE.csv'
nasdaq_filename = 'NASDAQ.csv'
hangseng_filename = 'HANG SENG.csv'
bovespa_filename = 'BOVESPA.csv'

window = 10
markowitz_window = 240
n_empresas = 5
num_simulations = 1000
num_simulations_markowitz = 1000
min_monkey_days = 2
money = 1000000

In [5]:
# Leemos los datos

dax_data = pd.read_csv(dax_filename, header=0,
                       parse_dates=True, index_col=0, sep=';', dayfirst=True)
ftse_data = pd.read_csv(ftse_filename, header=0,
                        parse_dates=True, index_col=0, sep=';', dayfirst=True)
nasdaq_data = pd.read_csv(nasdaq_filename, header=0,
                          parse_dates=True, index_col=0, sep=';', dayfirst=True)
hangseng_data = pd.read_csv(hangseng_filename, header=0,
                            parse_dates=True, index_col=0, sep=';', dayfirst=True)
bovespa_data = pd.read_csv(bovespa_filename, header=0,
                           parse_dates=True, index_col=0, sep=';', dayfirst=True)
curr_EUR = pd.DataFrame([1]*len(dax_data.index))
curr_GBP = pd.DataFrame(ftse_data.iloc[:, -1])
curr_USD = pd.DataFrame(nasdaq_data.iloc[:, :-1])
curr_HKD = pd.DataFrame(hangseng_data.iloc[:, :-1])
curr_BRL = pd.DataFrame(bovespa_data.iloc[:, :-1])

dax_data = dax_data.iloc[:, :-2]
ftse_data = ftse_data.iloc[:, :-1]
nasdaq_data = nasdaq_data.iloc[:, :-1]
hangseng_data = hangseng_data.iloc[:, :-1]
bovespa_data = bovespa_data.iloc[:, :-1]

dax_data = pd.DataFrame(
    dax_data.values * curr_EUR.values, index=dax_data.index, columns=dax_data.columns)
ftse_data = pd.DataFrame(
    ftse_data.values * curr_GBP.values, index=ftse_data.index, columns=ftse_data.columns)
nasdaq_data = pd.DataFrame(
    nasdaq_data.values * curr_USD.values, index=nasdaq_data.index, columns=nasdaq_data.columns)
hangseng_data = pd.DataFrame(
    hangseng_data.values * curr_HKD.values, index=hangseng_data.index, columns=hangseng_data.columns)
bovespa_data = pd.DataFrame(
    bovespa_data.values * curr_BRL.values, index=bovespa_data.index, columns=bovespa_data.columns)

benchmark_global = pd.concat(
    [dax_data, ftse_data, nasdaq_data, hangseng_data, bovespa_data], axis=1)

benchmark_global_ret = np.log(benchmark_global).diff()

In [6]:
def calcula_mono(monkey_df, n_companies, markowitz_window, min_monkey_days, num_simulations_markowitz):
    
    buy_date = sample(range(markowitz_window+1,
                            len(benchmark_global_ret.index)-min_monkey_days), 1)[0]
    sell_date = sample(
        range(buy_date+min_monkey_days, len(benchmark_global_ret.index)), 1)[0]
    
    temp_mark_df = benchmark_global_ret.iloc[(
        buy_date-markowitz_window):buy_date, :]
    
    ret_df = markowitz_random_companies_random_dates(
        temp_mark_df[:], n_empresas, num_simulations_markowitz)
    
    dates_df = pd.DataFrame.from_dict({'Buy_date': [buy_date],
                                       'Sell_date': [sell_date],
                                       })

    ret_df = pd.concat([ret_df, dates_df], axis=1)

    return ret_df

In [7]:
def calcula_mono_mult(monkey_df, n_companies, markowitz_window, min_monkey_days, num_simulations_markowitz, fool):
    
    buy_date = sample(range(markowitz_window+1,
                            len(benchmark_global_ret.index)-min_monkey_days), 1)[0]
    sell_date = sample(
        range(buy_date+min_monkey_days, len(benchmark_global_ret.index)), 1)[0]
    temp_mark_df = benchmark_global_ret.iloc[(
        buy_date-markowitz_window):buy_date, :]
    ret_df = markowitz_random_companies_random_dates(
        temp_mark_df[:], n_empresas, num_simulations_markowitz)
    dates_df = pd.DataFrame.from_dict({'Buy_date': [buy_date],
                                       'Sell_date': [sell_date],
                                       })

    ret_df = pd.concat([ret_df, dates_df], axis=1)

    return ret_df

In [8]:
def markowitz_random_companies_random_dates(mark_df, n_companies, num_simulations_markowitz):

    selected_companies = np.random.randint(len(mark_df.columns), size=(5))

    mark_df_ret = mark_df.iloc[:, selected_companies]
    cov_matrix = mark_df_ret.cov()

    daily_return = pd.DataFrame(mark_df_ret.sum(axis=0)
                                / (len(mark_df_ret.index)+1)).T

    # Calculamos los pesos y los normalizamos

    weights = pd.DataFrame(np.random.uniform(np.random.uniform(
        0.0, 100, size=(num_simulations, n_empresas))))
    weights_norm = weights.div(weights.sum(axis=1), axis=0)

    # Calculamos la rentabilidad de la cartera en función de los pesos
    weights_norm.columns = daily_return.columns
    temp_profit = daily_return @ weights_norm.T
    portfolio_profit = temp_profit.T.sum(axis=1)

    # Calculamos el riesgo de la cartera (desviación), en función de los pesos.
    portfolio_risk = (weights_norm @ cov_matrix).mul(
        weights_norm, axis='index').sum(axis=1)**.5

    # Calculamos la eficiencia de la cartera (pendiente), en función de los pesos.
    portfolio_efficiency = portfolio_profit.divide(portfolio_risk)

    # Localizamos la cartera con mayor rentabilidad, menor riesgo y mayor eficiencia.

    best_portfolio = portfolio_efficiency.argmax()

    zeros = np.zeros(shape=(1, len(benchmark_global.columns)))
    final_df = pd.DataFrame(zeros, columns=benchmark_global.columns)
    final_df.iloc[0, selected_companies] = weights_norm.iloc[best_portfolio, :]
    
    return final_df

In [9]:
monos_list = []

for _ in range(num_simulations):
    monos_list.append(calcula_mono(benchmark_global_ret, n_empresas,
                                   markowitz_window, min_monkey_days, num_simulations_markowitz))

results_df = pd.concat(monos_list)
results_df.index = range(num_simulations)
weights = results_df.iloc[:, :-2]
precios_compra = benchmark_global.iloc[results_df.iloc[:, -2], :]
precios_venta = benchmark_global.iloc[results_df.iloc[:, -1], :]
precios_compra.index = range(num_simulations)
precios_venta.index = range(num_simulations)

# results_df.to_csv('results_df.csv')
# precios_compra.to_csv('precios_compra.csv')
# precios_venta.to_csv('precios_venta.csv')

C:\Users\gmelendez\python-venv\venv-ml\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.


In [10]:
num_acciones = (weights.T).div(
    precios_compra.T, axis=0).T*money

rdo_accion = precios_venta.sub(
    precios_compra, axis=0).mul(num_acciones, axis=0)
rdo_simulacion = rdo_accion.sum(axis=1)
rentabilidad = np.log((rdo_simulacion+money)/money)

df_simulacion = pd.DataFrame(rdo_simulacion, index=range(
    num_simulations), columns=['rdo_simulacion'])
df_rentabilidad = pd.DataFrame(rentabilidad, index=range(
    num_simulations), columns=['rentabilidad'])

results_df = pd.concat(
    [results_df[:], df_simulacion, df_rentabilidad],  axis=1)

mejor_simulacion = results_df.iloc[:, -1].argmax()
quantile_list = list(range(11))
quantile_list = [x / 10 for x in quantile_list]
quantiles = results_df.iloc[:, -1].quantile(quantile_list)

C:\Users\gmelendez\python-venv\venv-ml\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.


In [11]:
quantiles

0.0   -1.584881
0.1   -0.288656
0.2   -0.103879
0.3    0.012444
0.4    0.084583
0.5    0.196107
0.6    0.349993
0.7    0.568595
0.8    0.849429
0.9    1.395972
1.0    5.080510
Name: rentabilidad, dtype: float64